In [15]:
pip install dotenv pypdf langchain langchain-community docx2txt tiktoken -q


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [7]:
import os
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv(), override=True)

True

In [8]:
pip install wikipedia -q 


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [9]:
def load_document(file):
    import os

    name, extension = os.path.splitext(file)

    if extension == ".pdf":
        from langchain_classic.document_loaders import PyPDFLoader

        print(f"Loading {file}")
        loader = PyPDFLoader(file)
    elif extension == ".docx":
        from langchain_classic.document_loaders import Docx2txtLoader

        loader = Docx2txtLoader(file)
    else:
        print("Document format is not supported")
        return None

    data = loader.load()
    return data


# wikipedia


def load_from_wikipedia(query, lang="en", load_max_docs=2):
    from langchain_classic.document_loaders import WikipediaLoader

    loader = WikipediaLoader(query=query, lang=lang, load_max_docs=load_max_docs)
    data = loader.load()
    return data

In [10]:
def chunk_data(data, chunk_size=256):
    from langchain_classic.text_splitter import RecursiveCharacterTextSplitter

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size, chunk_overlap=0
    )
    chunks = text_splitter.split_documents(data)
    return chunks

In [18]:
def print_embedding_cost(texts):
    import tiktoken

    enc = tiktoken.encoding_for_model("text-embedding-ada-002")
    total_tokens = sum([len(enc.encode(page.page_content)) for page in texts])
    print(f"Total Tokens: {total_tokens}")
    print(f"Embedding Cost in USD: {total_tokens / 1000 * 0.0004:.6f}")

## Running code

In [11]:
data = load_document("pdfs/guia_lgpd.pdf")
# print(data[1].page_content)

print(f"You have {len(data)} pages in your data")

Loading pdfs/guia_lgpd.pdf
You have 69 pages in your data


In [12]:
# data = load_from_wikipedia("Gemini-AI")
# print(data[0].page_content)

In [14]:
chunks = chunk_data(data)
print(len(chunks))
print(chunks[10].page_content)

929
HISTÓRICO DE VERSÕES
DATA VERSÃO DESCRIÇÃO AUTOR
23/03/2020 1.0 Primeira versão do Guia de Boas Práticas. Equipe Técnica de 
Elaboração
14/08/2020 2.0
Segunda versão do Guia de Boas Práticas 
considerando as sugestões enviadas pelo:


In [19]:
print_embedding_cost(chunks)

Total Tokens: 52383
Embedding Cost in USD: 0.020953
